In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np

2023-09-26 22:14:58.360302: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-26 22:14:58.833831: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
batch_size = 1024
l_rate = 0.001

In [3]:
@tf.keras.saving.register_keras_serializable()
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.dense1 = tf.keras.layers.Dense(units=128, activation=tf.nn.leaky_relu)
        self.dense2 = tf.keras.layers.Dense(units=1024, activation=tf.nn.leaky_relu)
        self.dense3 = tf.keras.layers.Dense(units=128, activation=tf.nn.leaky_relu)
        self.dense4 = tf.keras.layers.Dense(units=1024, activation=tf.nn.leaky_relu)
        self.dense5 = tf.keras.layers.Dense(units=8)

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        x = self.dense3(x)
        x = self.dense4(x)
        output = self.dense5(x)
        return output

In [4]:
def valiAll(index_epoch):
    y_v_p = model(X_v)
    va_mse = tf.reduce_mean(tf.square(y_v_p - y_v))
    va_rmse = tf.sqrt(va_mse)
    va_mae = tf.reduce_mean(tf.abs(y_v_p - y_v))
    va_r2 = 1 - tf.reduce_sum(tf.square(y_v_p - y_v)) / tf.reduce_sum(tf.square(y_v - tf.reduce_mean(y_v)))
    print("mse:{} rmse:{} mae:{} r2:{}".format(va_mse, va_rmse, va_mae, va_r2))

In [5]:
test_dataset = pd.read_csv("testset.csv", encoding='utf-8').sample(frac=1).reset_index(drop=True)
X_v = test_dataset.loc[:,'freq':'L2'].to_numpy(dtype = np.float32)
y_v = test_dataset.loc[:,'S11r':'S41i'].to_numpy(dtype = np.float32)

In [6]:
dataset1 = pd.read_csv('./20-24Trainset.csv', encoding='utf-8')
dataset2 = pd.read_csv('./50-54Trainset.csv', encoding='utf-8')
dataset = pd.concat([dataset1, dataset2], ignore_index=True).sample(frac=1).reset_index(drop=True)
X = dataset.loc[:,'freq':'L2'].to_numpy(dtype = np.float32)
y = dataset.loc[:,'S11r':'S41i'].to_numpy(dtype = np.float32)
dataset_train = tf.data.Dataset.from_tensor_slices((X, y))
dataset_train = dataset_train.shuffle(buffer_size=X.shape[0])
dataset_train = dataset_train.batch(batch_size)
dataset_train = dataset_train.prefetch(tf.data.experimental.AUTOTUNE)

2023-09-26 22:15:00.720059: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9604 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:17:00.0, compute capability: 7.5
2023-09-26 22:15:00.720587: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 9621 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:65:00.0, compute capability: 7.5


In [7]:
model = MLP()
optimizer = tf.keras.optimizers.Adam(learning_rate=l_rate)

In [8]:
for i in range(150):
    for X, y in dataset_train:
        with tf.GradientTape() as tape:
            y_pred = model(X)
            tr_mse = tf.reduce_mean(tf.square(y_pred - y))
        tr_rmse = tf.sqrt(tr_mse)
        tr_mae = tf.reduce_mean(tf.abs(y_pred - y))
        tr_r2 = 1 - tf.reduce_sum(tf.square(y_pred - y)) / tf.reduce_sum(tf.square(y - tf.reduce_mean(y)))
        grads = tape.gradient(tr_mse, model.variables)
        optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))
    print("epoch:{}".format(i))
    print("train mse:{} rmse:{} mae:{} r2:{}".format(tr_mse, tr_rmse, tr_mae, tr_r2))
    valiAll(i)

2023-09-26 22:15:01.766820: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55d4483d81b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-09-26 22:15:01.766840: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
2023-09-26 22:15:01.766844: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (1): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
2023-09-26 22:15:01.769880: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-09-26 22:15:01.874650: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-09-26 22:15:01.982119: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


epoch:0
train mse:0.04797578975558281 rmse:0.21903376281261444 mae:0.17104573547840118 r2:0.6074907779693604
mse:0.05394343286752701 rmse:0.23225724697113037 mae:0.18157859146595 r2:0.55818772315979
epoch:1
train mse:0.040751758962869644 rmse:0.2018706500530243 mae:0.15815585851669312 r2:0.6667999029159546
mse:0.04484250396490097 rmse:0.2117604911327362 mae:0.16445299983024597 r2:0.6327269673347473
epoch:2
train mse:0.0385105274617672 rmse:0.19624099135398865 mae:0.15314337611198425 r2:0.6832287311553955
mse:0.03858768939971924 rmse:0.19643749296665192 mae:0.151967853307724 r2:0.6839556694030762
epoch:3
train mse:0.040450114756822586 rmse:0.20112213492393494 mae:0.15411455929279327 r2:0.6688069105148315
mse:0.04238121211528778 rmse:0.2058669775724411 mae:0.15830989181995392 r2:0.6528856754302979
epoch:4
train mse:0.030904388055205345 rmse:0.17579643428325653 mae:0.13156168162822723 r2:0.7444779872894287
mse:0.034121349453926086 rmse:0.18471965193748474 mae:0.1395758092403412 r2:0.72053